# Star Wars People Query

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
# request people data
base_url = 'http://swapi.dev/api/people/?page='
i = 1
people = []
url = ''.join([base_url, str(i)])
request = requests.get(url).json()
while 'results' in request:
    people += request['results']
    i += 1
    url = ''.join([base_url, str(i)])
    request = requests.get(url).json() 

In [3]:
people[0]

{'name': 'Luke Skywalker',
 'height': '172',
 'mass': '77',
 'hair_color': 'blond',
 'skin_color': 'fair',
 'eye_color': 'blue',
 'birth_year': '19BBY',
 'gender': 'male',
 'homeworld': 'http://swapi.dev/api/planets/1/',
 'films': ['http://swapi.dev/api/films/1/',
  'http://swapi.dev/api/films/2/',
  'http://swapi.dev/api/films/3/',
  'http://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['http://swapi.dev/api/vehicles/14/',
  'http://swapi.dev/api/vehicles/30/'],
 'starships': ['http://swapi.dev/api/starships/12/',
  'http://swapi.dev/api/starships/22/'],
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'url': 'http://swapi.dev/api/people/1/'}

In [4]:
len(people)

82

## Pandas DataFrame

In [5]:
(
    max([len(p['films']) for p in people]), 
    max([len(p['species']) for p in people]),
    max([len(p['vehicles']) for p in people]),
    max([len(p['starships']) for p in people]),
)

(6, 1, 2, 5)

Some attributes like `films`, `species`, `vehicles`, `starships` is a list instead of a single element. Each person has only one species (or no species), so we can simplely take the element out of the list. For other attributes, we can make each one a single dataframe with `name`.

In [6]:
multiple_entries_col = ['films', 'vehicles', 'starships']

In [7]:
people_df = pd.json_normalize(people).drop(columns = multiple_entries_col)
people_df['species'] = people_df['species'].apply(lambda x: ''.join(x) if len(x) > 0 else 'n/a')
people_df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,http://swapi.dev/api/planets/1/,n/a,2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,http://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,http://swapi.dev/api/planets/1/,http://swapi.dev/api/species/2/,2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,http://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,http://swapi.dev/api/planets/8/,http://swapi.dev/api/species/2/,2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,http://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,http://swapi.dev/api/planets/1/,n/a,2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,http://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,http://swapi.dev/api/planets/2/,n/a,2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,http://swapi.dev/api/people/5/


In [8]:
film_name = pd.json_normalize(people, 'films', ['name']).rename(columns = {0:'films'})
film_name.head()

,films,name
0,http://swapi.dev/api/films/1/,Luke Skywalker
1,http://swapi.dev/api/films/2/,Luke Skywalker
2,http://swapi.dev/api/films/3/,Luke Skywalker
3,http://swapi.dev/api/films/6/,Luke Skywalker
4,http://swapi.dev/api/films/1/,C-3PO


In [9]:
vehicles_name = pd.json_normalize(people, 'vehicles', ['name']).rename(columns = {0:'vehicles'})
vehicles_name.head()

,vehicles,name
0,http://swapi.dev/api/vehicles/14/,Luke Skywalker
1,http://swapi.dev/api/vehicles/30/,Luke Skywalker
2,http://swapi.dev/api/vehicles/30/,Leia Organa
3,http://swapi.dev/api/vehicles/38/,Obi-Wan Kenobi
4,http://swapi.dev/api/vehicles/44/,Anakin Skywalker


In [10]:
starships_name = pd.json_normalize(people, 'starships', ['name']).rename(columns = {0:'starships'})
starships_name.head()

,starships,name
0,http://swapi.dev/api/starships/12/,Luke Skywalker
1,http://swapi.dev/api/starships/22/,Luke Skywalker
2,http://swapi.dev/api/starships/13/,Darth Vader
3,http://swapi.dev/api/starships/12/,Biggs Darklighter
4,http://swapi.dev/api/starships/48/,Obi-Wan Kenobi


We want to find the name of the oldest person (or robot or alien) and list the titles of all the films they appeared in. To find this information, we can use `birth_year` attribute. No death_year attribute, so we assume they are all alive. To find the oldest one, we just need to find the one that was borned earliest.
> **BBY**: The battle has become known as the Battle of Yavin, and its date is used as a place-marker for events in the expanded Star Wars universe. Events before the Battle of Yavin are marked by BBY ("Before the Battle of Yavin"), and events that occur after are marked by ABY ("After the Battle of Yavin")

In [11]:
[p['birth_year'] for p in people]

['19BBY',
 '112BBY',
 '33BBY',
 '41.9BBY',
 '19BBY',
 '52BBY',
 '47BBY',
 'unknown',
 '24BBY',
 '57BBY',
 '41.9BBY',
 '64BBY',
 '200BBY',
 '29BBY',
 '44BBY',
 '600BBY',
 '21BBY',
 'unknown',
 '896BBY',
 '82BBY',
 '31.5BBY',
 '15BBY',
 '53BBY',
 '31BBY',
 '37BBY',
 '41BBY',
 '48BBY',
 'unknown',
 '8BBY',
 'unknown',
 '92BBY',
 'unknown',
 '91BBY',
 '46BBY',
 '52BBY',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 '62BBY',
 '72BBY',
 '54BBY',
 'unknown',
 '48BBY',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 '72BBY',
 '92BBY',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 '22BBY',
 'unknown',
 'unknown',
 'unknown',
 '82BBY',
 'unknown',
 '58BBY',
 '40BBY',
 'unknown',
 '102BBY',
 '67BBY',
 '66BBY',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown']

In [12]:
people_film_df = people_df.merge(film_name)
people_film_df['birth_year'] = people_film_df['birth_year'].apply(lambda x: float(x[:-3]) if x != 'unknown' else np.nan)

In [13]:
people_film_df.nlargest(1, 'birth_year')

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,created,edited,url,films
58,Yoda,66,17,white,green,brown,896.0,male,http://swapi.dev/api/planets/28/,http://swapi.dev/api/species/6/,2014-12-15T12:26:01.042000Z,2014-12-20T21:17:50.345000Z,http://swapi.dev/api/people/20/,http://swapi.dev/api/films/2/


In [14]:
oldest_people_films = list(people_film_df[people_film_df.name == 'Yoda'].films)

In [15]:
films_titles = [requests.get(url).json()['title'] for url in oldest_people_films]

In [16]:
films_titles

['The Empire Strikes Back',
 'Return of the Jedi',
 'The Phantom Menace',
 'Attack of the Clones',
 'Revenge of the Sith']

***

In `species` resource, we can find the average lifespan of each species. I would also like to show people of species that has longest lifespan, and list the films they appeared in.

In [17]:
base_url = 'http://swapi.dev/api/species/?page='
i = 1
species = []
url = ''.join([base_url, str(i)])
request = requests.get(url).json()
while 'results' in request:
    species += request['results']
    i += 1
    url = ''.join([base_url, str(i)])
    request = requests.get(url).json() 

In [18]:
species[0]

{'name': 'Human',
 'classification': 'mammal',
 'designation': 'sentient',
 'average_height': '180',
 'skin_colors': 'caucasian, black, asian, hispanic',
 'hair_colors': 'blonde, brown, black, red',
 'eye_colors': 'brown, blue, green, hazel, grey, amber',
 'average_lifespan': '120',
 'homeworld': 'http://swapi.dev/api/planets/9/',
 'language': 'Galactic Basic',
 'people': ['http://swapi.dev/api/people/66/',
  'http://swapi.dev/api/people/67/',
  'http://swapi.dev/api/people/68/',
  'http://swapi.dev/api/people/74/'],
 'films': ['http://swapi.dev/api/films/1/',
  'http://swapi.dev/api/films/2/',
  'http://swapi.dev/api/films/3/',
  'http://swapi.dev/api/films/4/',
  'http://swapi.dev/api/films/5/',
  'http://swapi.dev/api/films/6/'],
 'created': '2014-12-10T13:52:11.567000Z',
 'edited': '2014-12-20T21:36:42.136000Z',
 'url': 'http://swapi.dev/api/species/1/'}

In [19]:
len(species)

37

In [20]:
species_df = pd.json_normalize(species).drop(columns = ['people', 'films'])
species_df['average_lifespan'] = (
    species_df['average_lifespan'].
    apply(lambda x: float(x) if x not in ['indefinite', 'unknown'] else np.nan)
)
species_df.head()

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,created,edited,url
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120.0,http://swapi.dev/api/planets/9/,Galactic Basic,2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,http://swapi.dev/api/species/1/
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,NaN,None,n/a,2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,http://swapi.dev/api/species/2/
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400.0,http://swapi.dev/api/planets/14/,Shyriiwook,2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,http://swapi.dev/api/species/3/
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,NaN,http://swapi.dev/api/planets/23/,Galatic Basic,2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,http://swapi.dev/api/species/4/
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000.0,http://swapi.dev/api/planets/24/,Huttese,2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,http://swapi.dev/api/species/5/


In [21]:
people_species = pd.json_normalize(species, 'people', ['name']).rename(columns = {0:'people'})
people_species.head()

,people,name
0,http://swapi.dev/api/people/66/,Human
1,http://swapi.dev/api/people/67/,Human
2,http://swapi.dev/api/people/68/,Human
3,http://swapi.dev/api/people/74/,Human
4,http://swapi.dev/api/people/2/,Droid


In [22]:
film_species = pd.json_normalize(species, 'films', ['name']).rename(columns = {0:'films'})
film_species.head()

,films,name
0,http://swapi.dev/api/films/1/,Human
1,http://swapi.dev/api/films/2/,Human
2,http://swapi.dev/api/films/3/,Human
3,http://swapi.dev/api/films/4/,Human
4,http://swapi.dev/api/films/5/,Human


In [23]:
species_df.nlargest(1, 'average_lifespan')

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,created,edited,url
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000.0,http://swapi.dev/api/planets/24/,Huttese,2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,http://swapi.dev/api/species/5/


In [24]:
p = (people_species[people_species.name == 'Hutt'].people)

In [25]:
p_name = [requests.get(url).json()['name'] for url in p]

In [26]:
p_name

['Jabba Desilijic Tiure']

In [27]:
f = list(film_species[film_species.name == 'Hutt'].films)

In [28]:
f_title = [requests.get(url).json()['title'] for url in f]

In [29]:
f_title

['A New Hope', 'Return of the Jedi']

## Mongdb query

In [30]:
from pymongo import MongoClient

In [31]:
client = MongoClient('mongodb:27017')
client.drop_database('starwars')
db = client.starwars

In [32]:
def get_nested(d):
    url = ''.join(d['species'])
    if len(url) > 0:
        d['species'] = requests.get(url).json()
    else:
        d['species'] = ''
        
    urls = d['films']
    starships = [requests.get(url).json() for url in urls]
    d['films']  = starships
    
    return d

In [33]:
[get_nested(p) for p in people]
pass

To make life eaiser, I trim the data to simplify the mongodb query. 

In [34]:
def trim_data(d):
    # keep only film title
    if len(d['films']) > 0:
        d['films_titles'] = [d['films'][i]['title'] for i in range(len(d['films']))]
    d['films'] = ''
    
    # transform birth year to float
    if d['birth_year'] != 'unknown':
        d['b_year'] = float(d['birth_year'][:-3])
        
    # transform avg lifespan of species to float
    if d['species'] != '':
        if d['species']['average_lifespan'] not in ['unknown', 'indefinite']:
            d['avg_lifespan'] = float(d['species']['average_lifespan'])
    d['species'] = ''
    
    return d

In [35]:
[trim_data(p) for p in people]
pass

In [36]:
people[0]

{'name': 'Luke Skywalker',
 'height': '172',
 'mass': '77',
 'hair_color': 'blond',
 'skin_color': 'fair',
 'eye_color': 'blue',
 'birth_year': '19BBY',
 'gender': 'male',
 'homeworld': 'http://swapi.dev/api/planets/1/',
 'films': '',
 'species': '',
 'vehicles': ['http://swapi.dev/api/vehicles/14/',
  'http://swapi.dev/api/vehicles/30/'],
 'starships': ['http://swapi.dev/api/starships/12/',
  'http://swapi.dev/api/starships/22/'],
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'url': 'http://swapi.dev/api/people/1/',
 'films_titles': ['A New Hope',
  'The Empire Strikes Back',
  'Return of the Jedi',
  'Revenge of the Sith'],
 'b_year': 19.0}

In [38]:
people_ = db.people_
result = people_.insert_many(people)

In [39]:
for x in people_.find(
    # search criteria
    {},
    # values to return
    {'_id': False,
     'name': True, 
     'films_titles': True
    }
).sort('b_year', -1).limit(1):
    print(x)

{'name': 'Yoda', 'films_titles': ['The Empire Strikes Back', 'Return of the Jedi', 'The Phantom Menace', 'Attack of the Clones', 'Revenge of the Sith']}


In [40]:
for x in people_.find(
    # search criteria
    {},
    # values to return
    {'_id': False,
     'name': True, 
     'films_titles': True
    }
).sort('avg_lifespan', -1).limit(1):
    print(x)

{'name': 'Jabba Desilijic Tiure', 'films_titles': ['A New Hope', 'Return of the Jedi', 'The Phantom Menace']}
